In [3]:
import pandas as pd
df = pd.read_csv('fuzzy_matched.csv')

- ใช้ความคล้ายของชื่อ species ทั้งหมด
- ใช้ fuzzy matching เทียบทุกชื่อ
- มี score_cutoff 80% (ยืดหยุ่นมาก)

ตัวอย่าง:
"Chonetinella andamanensis" อาจ match กับ
- "Chonetinella andamanensis" (100%)
- "Chonetinella sp." (85%)
- "Chonetinella cf. andamanensis" (90%)

In [4]:
import pandas as pd
from thefuzz import fuzz
from thefuzz import process

def fill_period_by_similarity(df):
    # สร้าง dictionary ของ species ที่มี PERIODFROM
    known_periods = df[df['PERIODFROM'].notna()].groupby('SCI_NAME')['PERIODFROM'].agg(lambda x: x.value_counts().index[0]).to_dict()
    
    # ฟังก์ชันหา most similar species
    def find_most_similar_species(species):
        if pd.isna(species):
            return None
        
        # หาชื่อ species ที่คล้ายที่สุดจาก known_periods
        matches = process.extractBests(
            species,
            known_periods.keys(),
            scorer=fuzz.token_sort_ratio,
            score_cutoff=80  # ความคล้ายขั้นต่ำ 80%
        )
        
        if matches:
            return known_periods[matches[0][0]]
        return None
    
    # เติม PERIODFROM สำหรับแถวที่เป็น NaN
    df_filled = df.copy()
    mask = df_filled['PERIODFROM'].isna()
    df_filled.loc[mask, 'PERIODFROM'] = df_filled.loc[mask, 'SCI_NAME'].apply(find_most_similar_species)
    
    return df_filled

# ตัวอย่างการใช้งาน
df_filled = fill_period_by_similarity(df)

In [5]:
A = df_filled[df_filled['SCI_NAME'].notnull() & df_filled['PERIODFROM'].notnull()][['SCI_NAME', 'PERIODFROM']]

In [6]:
def fill_period_by_taxonomy(df):
    # แยก genus จาก SCI_NAME
    df['GENUS'] = df['SCI_NAME'].str.split().str[0]
    
    # สร้าง dictionary ของ genus และ period
    genus_periods = df[df['PERIODFROM'].notna()].groupby('GENUS')['PERIODFROM'].agg(lambda x: x.value_counts().index[0]).to_dict()
    
    # เติม PERIODFROM โดยใช้ genus
    df_filled = df.copy()
    mask = df_filled['PERIODFROM'].isna()
    df_filled.loc[mask, 'PERIODFROM'] = df_filled.loc[mask, 'GENUS'].map(genus_periods)
    
    return df_filled

df_filled_by_taxonomy = fill_period_by_similarity(df)

In [7]:
B = df_filled_by_taxonomy[df_filled_by_taxonomy['SCI_NAME'].notnull() & df_filled_by_taxonomy['PERIODFROM'].notnull()][['SCI_NAME', 'PERIODFROM']]

In [8]:
compare = A == B
compare.sum()

SCI_NAME      9015
PERIODFROM    9015
dtype: int64

In [9]:
def fill_period_hybrid(df):
    # 1. ใช้ exact species match ก่อน
    species_periods = df[df['PERIODFROM'].notna()].groupby('SCI_NAME')['PERIODFROM'].agg(lambda x: x.value_counts().index[0]).to_dict()
    
    # 2. ใช้ similarity matching
    def get_period(row):
        if pd.isna(row['PERIODFROM']):
            # ตรวจสอบว่า SCI_NAME เป็น NaN หรือไม่
            if pd.isna(row['SCI_NAME']):
                return None
            
            # Check exact species match
            if row['SCI_NAME'] in species_periods:
                return species_periods[row['SCI_NAME']]
            
            try:
                # Check similarity
                matches = process.extractBests(
                    str(row['SCI_NAME']),  # แปลงเป็น string
                    [str(k) for k in species_periods.keys()],  # แปลง keys เป็น string
                    scorer=fuzz.token_sort_ratio,
                    score_cutoff=85
                )
                if matches:
                    return species_periods[matches[0][0]]
                
                # Check genus level
                if isinstance(row['SCI_NAME'], str) and ' ' in row['SCI_NAME']:
                    genus = row['SCI_NAME'].split()[0]
                    genus_matches = [s for s in species_periods.keys() if isinstance(s, str) and s.startswith(genus)]
                    if genus_matches:
                        return species_periods[genus_matches[0]]
            except Exception as e:
                print(f"Error processing row: {row['SCI_NAME']}, Error: {e}")
                return None
        
        return row['PERIODFROM']
    
    df_filled = df.copy()
    df_filled['PERIODFROM'] = df_filled.apply(get_period, axis=1)
    
    # Add confidence score
    df_filled['PERIOD_CONFIDENCE'] = df_filled.apply(
        lambda x: 1.0 if x['PERIODFROM'] == df.loc[x.name, 'PERIODFROM'] 
        else 0.7 if not pd.isna(x['PERIODFROM']) 
        else 0.0,
        axis=1
    )
    
    return df_filled

# ทดสอบการใช้งาน
try:
    df_filled = fill_period_hybrid(df)
    
    # แสดงผลลัพธ์
    print("\nSample of filled data:")
    print(df_filled[['SCI_NAME', 'PERIODFROM', 'PERIOD_CONFIDENCE']].head(10))
    
    # สรุปผล
    print("\nFilling summary:")
    print(f"Total rows: {len(df_filled)}")
    print(f"Filled rows: {df_filled['PERIODFROM'].notna().sum()}")
    print(f"Still missing: {df_filled['PERIODFROM'].isna().sum()}")
    
    # การกระจายของ confidence scores
    print("\nConfidence score distribution:")
    print(df_filled['PERIOD_CONFIDENCE'].value_counts())
    
except Exception as e:
    print(f"Error in main execution: {e}")

df_filled_by_hybrid = fill_period_hybrid(df)


Sample of filled data:
                    SCI_NAME  PERIODFROM  PERIOD_CONFIDENCE
0  Chonetinella andamanensis  เพอร์เมียน                0.7
1  Chonetinella andamanensis  เพอร์เมียน                1.0
2  Chonetinella andamanensis  เพอร์เมียน                1.0
3  Chonetinella andamanensis  เพอร์เมียน                1.0
4  Chonetinella andamanensis  เพอร์เมียน                1.0
5             Kasetia kaseti  เพอร์เมียน                0.7
6             Kasetia kaseti  เพอร์เมียน                1.0
7             Kasetia kaseti  เพอร์เมียน                1.0
8             Kasetia kaseti  เพอร์เมียน                1.0
9             Kasetia kaseti  เพอร์เมียน                1.0

Filling summary:
Total rows: 12016
Filled rows: 9856
Still missing: 2160

Confidence score distribution:
PERIOD_CONFIDENCE
0.7    7532
1.0    2324
0.0    2160
Name: count, dtype: int64


In [10]:
C = df_filled_by_hybrid[df_filled_by_hybrid['SCI_NAME'].notnull() & df_filled_by_hybrid['PERIODFROM'].notnull()][['SCI_NAME', 'PERIODFROM']]

In [11]:
A

,SCI_NAME,PERIODFROM
0,Chonetinella andamanensis,เพอร์เมียน
1,Chonetinella andamanensis,เพอร์เมียน
2,Chonetinella andamanensis,เพอร์เมียน
3,Chonetinella andamanensis,เพอร์เมียน
4,Chonetinella andamanensis,เพอร์เมียน
...,...,...
11923,Rusa unicolor,เพอร์เมียน
11924,Rusa unicolor,เพอร์เมียน
11925,Rusa unicolor,เพอร์เมียน
11972,Suidae indet.,เพอร์เมียนตอนต้น


In [21]:
B

,SCI_NAME,PERIODFROM
0,Chonetinella andamanensis,เพอร์เมียน
1,Chonetinella andamanensis,เพอร์เมียน
2,Chonetinella andamanensis,เพอร์เมียน
3,Chonetinella andamanensis,เพอร์เมียน
4,Chonetinella andamanensis,เพอร์เมียน
...,...,...
11923,Rusa unicolor,เพอร์เมียน
11924,Rusa unicolor,เพอร์เมียน
11925,Rusa unicolor,เพอร์เมียน
11972,Suidae indet.,เพอร์เมียนตอนต้น


In [12]:
C

,SCI_NAME,PERIODFROM
0,Chonetinella andamanensis,เพอร์เมียน
1,Chonetinella andamanensis,เพอร์เมียน
2,Chonetinella andamanensis,เพอร์เมียน
3,Chonetinella andamanensis,เพอร์เมียน
4,Chonetinella andamanensis,เพอร์เมียน
...,...,...
11923,Rusa unicolor,เพอร์เมียน
11924,Rusa unicolor,เพอร์เมียน
11925,Rusa unicolor,เพอร์เมียน
11972,Suidae indet.,เพอร์เมียนตอนต้น


### Eval Data Filled

In [13]:
# ตรวจสอบการกระจายตัวของ PERIODFROM
def analyze_period_distribution(df_original, df_filled):
    print("Original PERIODFROM distribution:")
    print(df_original['PERIODFROM'].value_counts(dropna=False))
    
    print("\nFilled PERIODFROM distribution:")
    print(df_filled['PERIODFROM'].value_counts(dropna=False))
    
    print("\nFill rate:", 
          (1 - df_filled['PERIODFROM'].isna().sum() / len(df_filled)) * 100,
          "%")

# ตรวจสอบความสมเหตุสมผล
def validate_filling(df_filled):
    # Check genus consistency
    genus_periods = df_filled.groupby(df_filled['SCI_NAME'].str.split().str[0])['PERIODFROM'].nunique()
    print("\nGenera with multiple periods:")
    print(genus_periods[genus_periods > 1])


# ตรวจสอบการกระจายตัวของ PERIODFROM
analyze_period_distribution(df, df_filled_by_hybrid)

Original PERIODFROM distribution:
PERIODFROM
NaN                                   9692
เพอร์เมียน                            1443
ครีเทเชียสตอนต้น                       491
เพอร์เมียนตอนต้น                       161
จูแรสสิกตอนปลาย                        138
ออร์โดวิเชียน                           35
ควอเทอร์นารี                            12
ไทรแอสสิกตอนปลาย                        11
จูแรสซิก                                 7
แคมเบรียนตอนปลาย                         5
ครีเทเซียสตอนต้น                         4
ออร์โดวีเชียน-ไซลูเรียน                  4
ไทรแอสสิก                                4
ยออร์โดวิเชียน                           3
จูแรสสิกตอนปลายถึงครีเทเชียสตอนต้น       2
ครีเทเชียส                               2
จูราสซิกตอนกลาง-ตอนปลาย                  1
ไทรแอสซิกตอนต้น                          1
Name: count, dtype: int64

Filled PERIODFROM distribution:
PERIODFROM
ครีเทเชียสตอนต้น                      5483
เพอร์เมียน                            3501
NaN                      

In [14]:
df_filled_by_hybrid

,FOSSIL_ID,SCI_NAME,COM_NAME,F_GROUP,F_TYPE,F_PART,DIS_NAME,PROVINCE_fossil,DISTRICT_fossil,TAMBOM,...,UPDATEDATE,GlobalID,OBJECTID,created_user,created_date,last_edited_user,last_edited_date,geometry_x,geometry_y,PERIOD_CONFIDENCE
0,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,NaN,{02C27880-6BA4-4CFD-97C6-BE452974953E},11247.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,455947.0,900155.0,0.7
1,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,NaN,{307AEFC8-3A20-4FB0-9454-2C4D84F80F61},11319.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,458554.0,897837.0,1.0
2,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,NaN,{8533F56F-4FA2-40D2-8376-7A585F5CB7CE},11320.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,456229.0,901537.0,1.0
3,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,NaN,{75F68926-73C3-4001-AD69-DDE48D55D316},11539.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,458300.0,901301.0,1.0
4,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,NaN,{00C780B7-477E-42D4-B547-60CD1C3779C1},11629.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,456951.0,893355.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12011,20230700044,Propotamochoerus cf. hysudricus,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,NaN,{DCDD5089-24F6-47F5-9F17-CF7D3EDFB25D},11246.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0
12012,20230700045,Propotamochoerus cf. hysudricus,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,NaN,{DCDD5089-24F6-47F5-9F17-CF7D3EDFB25D},11246.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0
12013,20230700046,Tetraconodontinae indet.,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,NaN,{DCDD5089-24F6-47F5-9F17-CF7D3EDFB25D},11246.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0
12014,20230700047,Tetraconodontinae indet.,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,NaN,{DCDD5089-24F6-47F5-9F17-CF7D3EDFB25D},11246.0,portaladmin,1.696235e+12,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0


In [15]:
len(df_filled_by_hybrid["PERIODFROM"].value_counts())

17

In [16]:
df_filled_by_hybrid["PERIODFROM"].unique()

array(['เพอร์เมียน', 'เพอร์เมียนตอนต้น', nan, None, 'ครีเทเชียสตอนต้น',
       'จูแรสสิกตอนปลาย', 'ควอเทอร์นารี', 'ยออร์โดวิเชียน', 'จูแรสซิก',
       'จูราสซิกตอนกลาง-ตอนปลาย', 'ไทรแอสสิกตอนปลาย', 'ครีเทเชียส',
       'ออร์โดวิเชียน', 'ไทรแอสสิก', 'ครีเทเซียสตอนต้น',
       'จูแรสสิกตอนปลายถึงครีเทเชียสตอนต้น', 'ไทรแอสซิกตอนต้น',
       'แคมเบรียนตอนปลาย', 'ออร์โดวีเชียน-ไซลูเรียน'], dtype=object)

In [17]:
df_filled_by_hybrid.to_csv('filled_by_hybrid.csv', index=False)

In [18]:
def analyze_nulls(df):
    # Calculate percentage of nulls for each column
    null_percentages = (df.isnull().sum() / len(df)) * 100
    
    # Categorize columns by null percentage
    high_nulls = null_percentages[null_percentages > 50]
    medium_nulls = null_percentages[(null_percentages > 20) & (null_percentages <= 50)]
    low_nulls = null_percentages[null_percentages <= 20]
    
    print("High null percentage (>50%):")
    print(high_nulls)
    print("\nMedium null percentage (20-50%):")
    print(medium_nulls)
    print("\nLow null percentage (<=20%):")
    print(low_nulls)

df_filled_by_hybrid = pd.read_csv('filled_by_hybrid.csv')
analyze_nulls(df_filled_by_hybrid)

High null percentage (>50%):
REGIS_CODE     88.390479
NAME_ENG      100.000000
LOCALITY       68.966378
FEATURE_TH     94.948402
FEATURE_EN    100.000000
GEO_DES_EN    100.000000
GEO_GROUP      88.199068
FORMATION      79.377497
PERIODTO      100.000000
DIS_BY_ENG    100.000000
DIS_BY_TH     100.000000
DIS_DAY       100.000000
DIS_MONTH     100.000000
DIS_YEAR      100.000000
OWNER_ENG     100.000000
POTENTIAL1    100.000000
POTENTIAL2    100.000000
POTENTIAL3    100.000000
POTENTIAL4    100.000000
POTENTIAL5    100.000000
POTENTIAL6    100.000000
POTENTIAL7    100.000000
STATUS        100.000000
UPDATEBY      100.000000
UPDATEDATE    100.000000
dtype: float64

Medium null percentage (20-50%):
GEO_DES_TH    43.858189
dtype: float64

Low null percentage (<=20%):
FOSSIL_ID             0.000000
SCI_NAME              3.844874
COM_NAME              0.000000
F_GROUP               0.000000
F_TYPE                0.000000
F_PART                0.000000
DIS_NAME              0.000000
PROVINCE_fo

In [19]:
df_filled_by_hybrid["FOS_GR_5"]

0        N
1        N
2        N
3        N
4        N
        ..
12011    N
12012    N
12013    N
12014    N
12015    N
Name: FOS_GR_5, Length: 12016, dtype: object

In [20]:
def handle_nulls_for_training(df):
    # 1. Drop columns with too many nulls
    cols_to_drop = [
        'NAME_ENG', 'FEATURE_EN', 'GEO_DES_EN', 
        'PERIODTO', 'DIS_BY_ENG', 'DIS_BY_TH',
        'POTENTIAL1', 'POTENTIAL2', 'POTENTIAL3',
        'STATUS', 'UPDATEBY', 'UPDATEDATE'
    ]
    
    # 2. Fill numeric features
    numeric_fills = {
        'UTM_E': df['UTM_E'].mean(),
        'UTM_N': df['UTM_N'].mean(),
        'geometry_x': df['geometry_x'].mean(),
        'geometry_y': df['geometry_y'].mean(),
        'SITE_CODE': -1  # Special value for missing
    }
    
    # 3. Fill categorical features
    categorical_fills = {
        'F_GROUP': 'Unknown',
        'FORMATION': 'Unknown',
        'GEO_GROUP': 'Unknown'
    }
    
    # 4. Handle text features
    text_fills = {
        'GEO_DES_TH': '',
        'FOS_DES_TH': '',
        'LOCALITY': ''
    }
    
    df_processed = df.copy()
    df_processed.drop(columns=cols_to_drop, inplace=True)
    df_processed.fillna(numeric_fills, inplace=True)
    df_processed.fillna(categorical_fills, inplace=True)
    df_processed.fillna(text_fills, inplace=True)
    
    # 5. Add missing indicators
    for col in numeric_fills.keys():
        df_processed[f'{col}_is_missing'] = df[col].isnull().astype(int)
    
    return df_processed

df_processed = handle_nulls_for_training(df_filled_by_hybrid)
df_processed

,FOSSIL_ID,SCI_NAME,COM_NAME,F_GROUP,F_TYPE,F_PART,DIS_NAME,PROVINCE_fossil,DISTRICT_fossil,TAMBOM,...,last_edited_user,last_edited_date,geometry_x,geometry_y,PERIOD_CONFIDENCE,UTM_E_is_missing,UTM_N_is_missing,geometry_x_is_missing,geometry_y_is_missing,SITE_CODE_is_missing
0,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,DMRAPP,1.696266e+12,455947.0,900155.0,0.7,0,0,0,0,0
1,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,DMRAPP,1.696266e+12,458554.0,897837.0,1.0,0,0,0,0,0
2,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,DMRAPP,1.696266e+12,456229.0,901537.0,1.0,0,0,0,0,0
3,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,DMRAPP,1.696266e+12,458300.0,901301.0,1.0,0,0,0,0,0
4,20200800005,Chonetinella andamanensis,แบรคิโอพอด,Invertebrate,แบรคิโอพอด,Dorsal internal mould and ventral external mou...,คณะวิจัยร่วม การลำดับชั้นหินและบรรพชีวินวิทยาโ...,พังงา,เกาะยาว,เกาะยาวใหญ่,...,DMRAPP,1.696266e+12,456951.0,893355.0,1.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12011,20230700044,Propotamochoerus cf. hysudricus,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0,0,0,0,0,0
12012,20230700045,Propotamochoerus cf. hysudricus,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0,0,0,0,0,0
12013,20230700046,Tetraconodontinae indet.,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0,0,0,0,0,0
12014,20230700047,Tetraconodontinae indet.,หมู,Vertebrate,สัตว์กีบคู่,ฟัน,คณะสำรวจวิจัยโบราณชีววิทยาไทย-ฝรั่งเศส,พะเยา,เชียงม่วน,สระ,...,DMRAPP,1.696266e+12,630558.0,2094844.0,0.0,0,0,0,0,0
